# Planner Agent for Multi-Agent HR Analytics System

This notebook implements a **Planner Agent** that:
1. Analyzes user questions to understand intent
2. Classifies questions as **"WHAT"** or **"WHY"** questions
3. Routes to appropriate agents:
   - **WHAT questions** → Text-to-SQL Agent (EDA) + Visualization Agent
   - **WHY questions** → Hypothesis Agent + Statistical Testing Agent

**Flow:** `User Question → Planner Agent → Route to Appropriate Agents → Return Results`

## 1. Setup Environment & Dependencies

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
from typing import Dict, Any, Literal

# Load environment variables
load_dotenv(override=True)

print("✅ Environment loaded")

✅ Environment loaded


## 2. Initialize LLM

In [2]:
# Initialize LLM pointing to LM Studio
llm = ChatOpenAI(
    model=os.environ.get("OPENAI_MODEL", "ibm/granite-3.2-8b"),
    base_url=os.environ.get("OPENAI_BASE_URL", "http://127.0.0.1:1234/v1"),
    api_key=os.environ.get("OPENAI_API_KEY", "lm-studio"),
    temperature=0.0,
)

print("✅ LLM initialized:", os.environ.get("OPENAI_MODEL", "ibm/granite-3.2-8b"))

✅ LLM initialized: ibm/granite-3.2-8b


## 3. Define HR Analytics Context

In [3]:
# HR Analytics Domain Context
HR_CONTEXT = """
DOMAIN: HR Employee Attrition Analytics

AVAILABLE DATA:
- Employee demographics (age, gender, marital status)
- Job information (department, role, level, salary)
- Work-life factors (overtime, business travel, distance from home)
- Satisfaction metrics (job, environment, relationship, work-life balance)
- Career progression (years at company, promotions, training)
- Attrition status (whether employee left the company)

ANALYSIS CAPABILITIES:
1. Descriptive Analytics (WHAT): Counts, averages, distributions, comparisons
2. Causal Analytics (WHY): Hypothesis testing, statistical significance, root cause analysis
"""

print("✅ HR Analytics context defined")

✅ HR Analytics context defined


## 4. Planner Agent Implementation

In [4]:
class PlannerAgent:
    """
    Planner Agent for Multi-Agent HR Analytics System
    
    Analyzes user questions and routes to appropriate agents:
    - WHAT questions → Text-to-SQL + Visualization
    - WHY questions → Hypothesis + Statistical Testing
    """
    
    def __init__(self, llm):
        """
        Initialize the Planner Agent.
        
        Args:
            llm: LangChain ChatOpenAI instance
        """
        self.llm = llm
        self._setup_prompt()
    
    def _setup_prompt(self):
        """Setup the planning prompt for question classification."""
        self.prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are an EXPERT HR Analytics Planner Agent. Your job is to analyze user questions and "
             "determine the appropriate analytical approach.\n\n"
             
             "═══════════════════════════════════════════════════════════\n"
             "CRITICAL TASK:\n"
             "═══════════════════════════════════════════════════════════\n"
             "Analyze the user's question and classify it into ONE of two categories:\n\n"
             
             "1. **WHAT Questions** (Descriptive Analytics):\n"
             "   - Asking for FACTS, COUNTS, AVERAGES, DISTRIBUTIONS\n"
             "   - Examples:\n"
             "     * \"What is the attrition rate?\"\n"
             "     * \"How many employees are in each department?\"\n"
             "     * \"What is the average salary by job role?\"\n"
             "     * \"Show me the distribution of overtime workers\"\n"
             "     * \"Compare attrition rates between departments\"\n"
             "   - Keywords: what, how many, show, list, compare, distribution, average, count\n"
             "   - **Route to:** Text-to-SQL Agent (EDA) + Visualization Agent\n\n"
             
             "2. **WHY Questions** (Causal Analytics):\n"
             "   - Asking for REASONS, CAUSES, EXPLANATIONS, RELATIONSHIPS\n"
             "   - Examples:\n"
             "     * \"Why do employees leave?\"\n"
             "     * \"What causes high attrition?\"\n"
             "     * \"Does overtime affect attrition?\"\n"
             "     * \"Is there a relationship between satisfaction and turnover?\"\n"
             "     * \"Why do male employees have higher attrition?\"\n"
             "   - Keywords: why, cause, reason, affect, impact, relationship, correlation, influence\n"
             "   - **Route to:** Hypothesis Agent + Statistical Testing Agent\n\n"
             
             "═══════════════════════════════════════════════════════════\n"
             "CLASSIFICATION RULES:\n"
             "═══════════════════════════════════════════════════════════\n"
             "1. If the question asks \"WHAT is/are\", \"HOW MANY\", \"SHOW ME\" → WHAT\n"
             "2. If the question asks \"WHY\", \"WHAT CAUSES\", \"DOES X AFFECT Y\" → WHY\n"
             "3. If the question asks for COMPARISON without causation → WHAT\n"
             "4. If the question asks about IMPACT or RELATIONSHIP → WHY\n"
             "5. If the question mentions HYPOTHESIS or TESTING → WHY\n"
             "6. If unclear, default to WHAT (descriptive is safer)\n\n"
             
             "═══════════════════════════════════════════════════════════\n"
             "OUTPUT FORMAT (JSON):\n"
             "═══════════════════════════════════════════════════════════\n"
             "Return ONLY a valid JSON object with this EXACT structure:\n"
             "{{\n"
             "  \"question_type\": \"WHAT\" or \"WHY\",\n"
             "  \"reasoning\": \"Brief explanation of why you classified it this way\",\n"
             "  \"agents_to_call\": [\"list\", \"of\", \"agents\"],\n"
             "  \"analysis_approach\": \"Short description of the analytical approach\"\n"
             "}}\n\n"
             
             "Agents available:\n"
             "- \"text_to_sql\" (EDA - retrieves data)\n"
             "- \"visualization\" (creates charts)\n"
             "- \"hypothesis\" (generates testable hypotheses)\n"
             "- \"statistical_testing\" (performs statistical tests)\n\n"
             
             "═══════════════════════════════════════════════════════════\n"
             "HR ANALYTICS CONTEXT:\n"
             "═══════════════════════════════════════════════════════════\n"
             "{hr_context}\n\n"
             
             "═══════════════════════════════════════════════════════════\n"
             "REMEMBER:\n"
             "═══════════════════════════════════════════════════════════\n"
             "- Return ONLY valid JSON\n"
             "- No markdown code blocks\n"
             "- No explanations outside the JSON\n"
             "- Be decisive - choose WHAT or WHY\n"
             "═══════════════════════════════════════════════════════════\n"),
            ("user",
             "Analyze this question and provide the routing decision:\n\n"
             "Question: {question}\n\n"
             "Return your analysis as JSON.")
        ])
        
        self.chain = self.prompt | self.llm | StrOutputParser()
    
    def _parse_json_response(self, response: str) -> Dict[str, Any]:
        """Parse JSON from LLM response, handling markdown code blocks."""
        import re
        
        # Try to extract from ```json ... ``` blocks
        json_block = re.search(r"```json\s*(.*?)```", response, re.IGNORECASE | re.DOTALL)
        if json_block:
            json_str = json_block.group(1)
        else:
            # Try generic ``` ... ``` blocks
            code_block = re.search(r"```\s*(.*?)```", response, re.DOTALL)
            json_str = code_block.group(1) if code_block else response
        
        # Parse JSON
        try:
            return json.loads(json_str.strip())
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON response: {e}\n\nResponse: {response}")
    
    def analyze_question(self, question: str, verbose: bool = True) -> Dict[str, Any]:
        """
        Analyze user question and determine routing.
        
        Args:
            question: User's natural language question
            verbose: If True, print analysis details
            
        Returns:
            Dictionary with routing decision:
            {
                'question_type': 'WHAT' or 'WHY',
                'reasoning': 'explanation',
                'agents_to_call': ['agent1', 'agent2'],
                'analysis_approach': 'description'
            }
        """
        # Get LLM response
        response = self.chain.invoke({
            "hr_context": HR_CONTEXT,
            "question": question
        })
        
        # Parse JSON
        decision = self._parse_json_response(response)
        
        if verbose:
            self._print_analysis(question, decision)
        
        return decision
    
    def _print_analysis(self, question: str, decision: Dict[str, Any]):
        """Pretty print the analysis results."""
        print("\n" + "═" * 70)
        print("🤔 PLANNER AGENT ANALYSIS")
        print("═" * 70)
        print(f"\n📝 User Question: {question}")
        print(f"\n🎯 Question Type: {decision['question_type']}")
        print(f"\n💭 Reasoning: {decision['reasoning']}")
        print(f"\n🤖 Agents to Call: {', '.join(decision['agents_to_call'])}")
        print(f"\n📊 Analysis Approach: {decision['analysis_approach']}")
        print("\n" + "═" * 70 + "\n")
    
    def route_question(self, question: str, verbose: bool = True) -> str:
        """
        Simple routing function that returns the question type.
        
        Args:
            question: User's question
            verbose: If True, print analysis
            
        Returns:
            'WHAT' or 'WHY'
        """
        decision = self.analyze_question(question, verbose=verbose)
        return decision['question_type']

print("✅ PlannerAgent class defined")

✅ PlannerAgent class defined


## 5. Initialize Planner Agent

In [5]:
# Initialize the Planner Agent
planner = PlannerAgent(llm=llm)

print("✅ Planner Agent initialized and ready!")

✅ Planner Agent initialized and ready!


## 6. Test with WHAT Questions (Descriptive Analytics)

In [6]:
# Test 1: Simple WHAT question
question1 = "What is the attrition rate in the company?"
result1 = planner.analyze_question(question1)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: What is the attrition rate in the company?

🎯 Question Type: WHAT

💭 Reasoning: The question asks for a factual figure, specifically the attrition rate, which falls under descriptive analytics.

🤖 Agents to Call: text_to_sql

📊 Analysis Approach: Descriptive analysis to calculate and present the attrition rate.

══════════════════════════════════════════════════════════════════════



In [7]:
# Test 2: Comparison WHAT question
question2 = "How many employees work overtime vs non-overtime?"
result2 = planner.analyze_question(question2)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: How many employees work overtime vs non-overtime?

🎯 Question Type: WHAT

💭 Reasoning: The question asks for a count or comparison of two groups (overtime and non-overtime workers), which falls under descriptive analytics.

🤖 Agents to Call: text_to_sql

📊 Analysis Approach: Retrieve data on overtime status to calculate the number of employees in each group.

══════════════════════════════════════════════════════════════════════



In [8]:
# Test 3: Distribution WHAT question
question3 = "Show me the average salary by department"
result3 = planner.analyze_question(question3)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Show me the average salary by department

🎯 Question Type: WHAT

💭 Reasoning: The question asks for a specific numerical value (average) related to a categorized group (department). This falls under descriptive analytics, seeking facts and counts.

🤖 Agents to Call: text_to_sql

📊 Analysis Approach: Retrieve the average salary data grouped by department using SQL queries.

══════════════════════════════════════════════════════════════════════



In [9]:
# Test 4: Comparison across groups
question4 = "Compare attrition rates between male and female employees"
result4 = planner.analyze_question(question4)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Compare attrition rates between male and female employees

🎯 Question Type: WHAT

💭 Reasoning: The question asks for a comparison of attrition rates, which is a descriptive analysis focusing on facts and differences.

🤖 Agents to Call: text_to_sql

📊 Analysis Approach: This involves retrieving data on attrition rates for male and female employees and performing a side-by-side comparison.

══════════════════════════════════════════════════════════════════════



## 7. Test with WHY Questions (Causal Analytics)

In [10]:
# Test 5: Direct WHY question
question5 = "Why do employees leave the company?"
result5 = planner.analyze_question(question5)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Why do employees leave the company?

🎯 Question Type: WHY

💭 Reasoning: The question asks for reasons or causes, which falls under causal analytics.

🤖 Agents to Call: hypothesis, statistical_testing

📊 Analysis Approach: This involves generating hypotheses about factors contributing to attrition and performing statistical tests to determine their significance.

══════════════════════════════════════════════════════════════════════



In [11]:
# Test 6: Causation question
question6 = "Does overtime work affect employee attrition?"
result6 = planner.analyze_question(question6)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Does overtime work affect employee attrition?

🎯 Question Type: WHY

💭 Reasoning: The question asks for a cause-effect relationship, specifically if overtime work has an impact on employee attrition.

🤖 Agents to Call: hypothesis, statistical_testing

📊 Analysis Approach: This is a causal query that requires generating hypotheses and performing statistical tests to determine the relationship between overtime work and employee attrition.

══════════════════════════════════════════════════════════════════════



In [12]:
# Test 7: Relationship question
question7 = "Is there a relationship between job satisfaction and attrition?"
result7 = planner.analyze_question(question7)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Is there a relationship between job satisfaction and attrition?

🎯 Question Type: WHY

💭 Reasoning: The question asks for an explanation of the relationship between two variables, implying causation or correlation.

🤖 Agents to Call: hypothesis, statistical_testing

📊 Analysis Approach: This requires generating a hypothesis and performing statistical tests to determine if there's a significant relationship between job satisfaction and attrition.

══════════════════════════════════════════════════════════════════════



In [13]:
# Test 8: Impact question
question8 = "What factors impact employee turnover the most?"
result8 = planner.analyze_question(question8)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: What factors impact employee turnover the most?

🎯 Question Type: WHY

💭 Reasoning: The question asks for reasons or causes of employee turnover, which falls under causal analytics.

🤖 Agents to Call: hypothesis, statistical_testing

📊 Analysis Approach: This involves generating hypotheses about potential factors influencing turnover and then performing statistical tests to determine the significance and strength of these relationships.

══════════════════════════════════════════════════════════════════════



## 8. Test Edge Cases

In [14]:
# Test 9: Ambiguous question (should default to WHAT)
question9 = "Tell me about employee attrition"
result9 = planner.analyze_question(question9)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Tell me about employee attrition

🎯 Question Type: WHAT

💭 Reasoning: The question asks for information about employee attrition, which is a descriptive inquiry into facts and counts.

🤖 Agents to Call: text_to_sql, visualization

📊 Analysis Approach: Descriptive analysis to retrieve and visualize data on employee attrition rates, trends, and patterns.

══════════════════════════════════════════════════════════════════════



In [15]:
# Test 10: Mixed question
question10 = "What is the attrition rate and why is it so high?"
result10 = planner.analyze_question(question10)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: What is the attrition rate and why is it so high?

🎯 Question Type: WHY

💭 Reasoning: The question asks for both a fact (attrition rate) and a reason (why it's high), indicating a causal interest.

🤖 Agents to Call: hypothesis, statistical_testing

📊 Analysis Approach: This question requires understanding the causes of high attrition, necessitating hypothesis generation and statistical testing to uncover relationships or correlations.

══════════════════════════════════════════════════════════════════════



## 9. Batch Testing

In [16]:
# Test multiple questions and summarize results
test_questions = [
    "What is the average age of employees?",
    "Why do young employees have higher attrition?",
    "How many employees are in the Sales department?",
    "Does work-life balance impact retention?",
    "Show me the distribution of job satisfaction scores",
    "What causes low job satisfaction?",
    "Compare monthly income across departments",
    "Is there a correlation between distance from home and attrition?"
]

results_summary = []

for q in test_questions:
    result = planner.analyze_question(q, verbose=False)
    results_summary.append({
        'question': q,
        'type': result['question_type'],
        'agents': ', '.join(result['agents_to_call'])
    })

# Display results as a table
import pandas as pd
df_summary = pd.DataFrame(results_summary)
print("\n📊 BATCH TEST RESULTS:")
print("=" * 100)
display(df_summary)


📊 BATCH TEST RESULTS:


,question,type,agents
0,What is the average age of employees?,WHAT,text_to_sql
1,Why do young employees have higher attrition?,WHY,"hypothesis, statistical_testing"
2,How many employees are in the Sales department?,WHAT,text_to_sql
3,Does work-life balance impact retention?,WHY,"hypothesis, statistical_testing"
4,Show me the distribution of job satisfaction s...,WHAT,visualization
5,What causes low job satisfaction?,WHY,"hypothesis, statistical_testing"
6,Compare monthly income across departments,WHAT,"text_to_sql, visualization"
7,Is there a correlation between distance from h...,WHY,"hypothesis, statistical_testing"


## 10. Integration with Other Agents (Mock Example)

Here's how the Planner Agent would integrate with the full system:

In [17]:
def process_user_question(question: str):
    """
    Complete pipeline with Planner Agent routing.
    
    This is a mock example showing how to integrate with other agents.
    """
    # Step 1: Analyze question with Planner
    decision = planner.analyze_question(question, verbose=True)
    
    # Step 2: Route to appropriate agents
    if decision['question_type'] == 'WHAT':
        print("\n🔄 Routing to: Text-to-SQL Agent + Visualization Agent\n")
        # Mock: In real implementation, call text_to_sql_agent and viz_agent
        print("[MOCK] Calling Text-to-SQL Agent...")
        print("[MOCK] Generating SQL query...")
        print("[MOCK] Executing query...")
        print("[MOCK] Creating visualization...")
        print("\n✅ Results: DataFrame + Interactive Chart")
        
    elif decision['question_type'] == 'WHY':
        print("\n🔄 Routing to: Hypothesis Agent + Statistical Testing Agent\n")
        # Mock: In real implementation, call hypothesis_agent and stats_agent
        print("[MOCK] Calling Hypothesis Agent...")
        print("[MOCK] Generating hypotheses...")
        print("[MOCK] Calling Statistical Testing Agent...")
        print("[MOCK] Performing chi-square/t-tests...")
        print("\n✅ Results: Hypotheses + Statistical Test Results")
    
    return decision

print("✅ Integration function 'process_user_question()' defined")

✅ Integration function 'process_user_question()' defined


## 11. Test Complete Pipeline (Mock)

In [18]:
# Test WHAT question routing
process_user_question("What is the attrition rate by department?")


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: What is the attrition rate by department?

🎯 Question Type: WHAT

💭 Reasoning: The question asks for a specific fact (attrition rate) broken down by a categorical variable (department). This falls under descriptive analytics, seeking to understand the distribution of attrition across different departments.

🤖 Agents to Call: text_to_sql

📊 Analysis Approach: Descriptive analysis using SQL queries to extract and summarize attrition data by department.

══════════════════════════════════════════════════════════════════════


🔄 Routing to: Text-to-SQL Agent + Visualization Agent

[MOCK] Calling Text-to-SQL Agent...
[MOCK] Generating SQL query...
[MOCK] Executing query...
[MOCK] Creating visualization...

✅ Results: DataFrame + Interactive Chart


{'question_type': 'WHAT',
 'reasoning': 'The question asks for a specific fact (attrition rate) broken down by a categorical variable (department). This falls under descriptive analytics, seeking to understand the distribution of attrition across different departments.',
 'agents_to_call': ['text_to_sql'],
 'analysis_approach': 'Descriptive analysis using SQL queries to extract and summarize attrition data by department.'}

In [19]:
# Test WHY question routing
process_user_question("Why do employees in Sales have higher attrition?")


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Why do employees in Sales have higher attrition?

🎯 Question Type: WHY

💭 Reasoning: The question asks for a reason or cause, specifically why employees in the Sales department have higher attrition.

🤖 Agents to Call: hypothesis, statistical_testing

📊 Analysis Approach: This involves generating hypotheses about potential factors contributing to higher attrition in the Sales department and then testing these hypotheses using statistical methods.

══════════════════════════════════════════════════════════════════════


🔄 Routing to: Hypothesis Agent + Statistical Testing Agent

[MOCK] Calling Hypothesis Agent...
[MOCK] Generating hypotheses...
[MOCK] Calling Statistical Testing Agent...
[MOCK] Performing chi-square/t-tests...

✅ Results: Hypotheses + Statistical Test Results


{'question_type': 'WHY',
 'reasoning': 'The question asks for a reason or cause, specifically why employees in the Sales department have higher attrition.',
 'agents_to_call': ['hypothesis', 'statistical_testing'],
 'analysis_approach': 'This involves generating hypotheses about potential factors contributing to higher attrition in the Sales department and then testing these hypotheses using statistical methods.'}

## 12. Try Your Own Question

In [ ]:
# Enter your question here
my_question = "Your question here"
process_user_question(my_question)


══════════════════════════════════════════════════════════════════════
🤔 PLANNER AGENT ANALYSIS
══════════════════════════════════════════════════════════════════════

📝 User Question: Your question here

🎯 Question Type: WHAT

💭 Reasoning: The question asks for a specific fact (the number of employees in each department), which falls under descriptive analytics.

🤖 Agents to Call: text_to_sql

📊 Analysis Approach: Descriptive analysis to retrieve and summarize the count of employees in each department.

══════════════════════════════════════════════════════════════════════


🔄 Routing to: Text-to-SQL Agent + Visualization Agent

[MOCK] Calling Text-to-SQL Agent...
[MOCK] Generating SQL query...
[MOCK] Executing query...
[MOCK] Creating visualization...

✅ Results: DataFrame + Interactive Chart


{'question_type': 'WHAT',
 'reasoning': 'The question asks for a specific fact (the number of employees in each department), which falls under descriptive analytics.',
 'agents_to_call': ['text_to_sql'],
 'analysis_approach': 'Descriptive analysis to retrieve and summarize the count of employees in each department.'}

: 

## 13. Summary & Usage Guide

### 🎯 What the Planner Agent Does

The Planner Agent is the **orchestrator** of the multi-agent system:

1. **Analyzes** user questions to understand intent
2. **Classifies** questions as WHAT (descriptive) or WHY (causal)
3. **Routes** to appropriate specialized agents
4. **Returns** a structured decision with reasoning

### 📊 Classification Logic

**WHAT Questions** → Text-to-SQL + Visualization:
- Asking for facts, counts, averages, distributions
- Keywords: what, how many, show, compare, list
- Example: "What is the average salary by department?"

**WHY Questions** → Hypothesis + Statistical Testing:
- Asking for reasons, causes, relationships, impact
- Keywords: why, cause, affect, impact, relationship, correlation
- Example: "Why do employees leave the company?"

### 🚀 Usage

```python
# Simple routing
question_type = planner.route_question("Your question")
# Returns: 'WHAT' or 'WHY'

# Detailed analysis
decision = planner.analyze_question("Your question")
# Returns: {
#   'question_type': 'WHAT' or 'WHY',
#   'reasoning': 'explanation',
#   'agents_to_call': ['agent1', 'agent2'],
#   'analysis_approach': 'description'
# }

# Complete pipeline
process_user_question("Your question")
# Analyzes and routes to appropriate agents
```

### ✨ Key Features

- ✅ **Intelligent routing** based on question intent
- ✅ **Structured JSON output** for easy integration
- ✅ **Detailed reasoning** for transparency
- ✅ **Domain-aware** with HR analytics context
- ✅ **Handles edge cases** with sensible defaults

### 🔗 Integration with Other Agents

```python
# Pseudo-code for full integration
decision = planner.analyze_question(user_question)

if decision['question_type'] == 'WHAT':
    df = text_to_sql_agent.query(user_question)
    fig = viz_agent.visualize(df, user_question)
    return df, fig
    
elif decision['question_type'] == 'WHY':
    hypotheses = hypothesis_agent.generate(user_question)
    test_results = stats_agent.test(hypotheses)
    return hypotheses, test_results
```

### 🎓 Next Steps

1. Test with your own questions
2. Integrate with existing Text-to-SQL Agent
3. Connect with Hypothesis and Statistical Testing Agents
4. Build a unified interface that uses the planner for all questions
5. Add logging and monitoring for question routing decisions